# Hello, Pinecone!

This notebook will walk through the steps to get a simple Pinecone index up and running. 


## Prerequisites

Install dependencies.

In [1]:
!pip install -qU pip pinecone-client

Set up Pinecone. Get your Pinecone API key [here](https://www.pinecone.io/start).

In [1]:
import pinecone
import os

api_key = os.getenv("PINECONE_API_KEY") or "USE_YOUR_API_KEY"
pinecone.init(api_key=api_key, environment='internal-beta')

Check Pinecone version compatibility.

In [2]:
import pinecone.info

version_info = pinecone.info.version()
server_version = ".".join(version_info.server.split(".")[:2])
client_version = ".".join(version_info.client.split(".")[:2])
notebook_version = "0.10"

assert (
    notebook_version == server_version
), "This notebook is outdated. Consider using the latest version of the notebook."
assert client_version == server_version, "Please upgrade pinecone-client."

## Pinecone quickstart

With Pinecone you can create a vector index where you can store and search through your vectors. 

In [3]:
# Giving our index a name
index_name = "hello-pinecone"
# Delete the index, if an index of the same name already exists
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

Creating a Pinecone Index.

In [4]:
dimensions = 3
pinecone.create_index(name=index_name, dimension=dimensions, metric="cosine", shards=1)

In [5]:
index = pinecone.Index(index_name=index_name)

We have the index ready. Now we will create some simple vectors that will serve as our examples.

In [6]:
import pandas as pd

df = pd.DataFrame(
    data={
        "id": ["A", "B"], 
        "vector": [[1., 1., 1.], [1., 2., 3.]]
    })
df

,id,vector
0,A,"[1.0, 1.0, 1.0]"
1,B,"[1.0, 2.0, 3.0]"


We perform upsert operations in our index. This call will insert a new vector in the index or update the vector if the id was already present.

In [7]:
index.upsert(vectors=zip(df.id, df.vector))  # insert vectors

{}

In [8]:
index.summarize()

{'dimension': 3, 'namespaces': {'': {'vector_count': 2}}}

In [9]:
index.query(
    queries=[[2., 2., 2.], [2., 4., 6.]], 
    top_k=5, 
    include_values=True) # returns top_k matches for every query

{'results': [{'matches': [{'id': 'A',
                           'score': 0.99999994,
                           'values': [0.99999994, 0.99999994, 0.99999994]},
                          {'id': 'B',
                           'score': 0.925820053,
                           'values': [1.0, 2.0, 3.0]}],
              'namespace': ''},
             {'matches': [{'id': 'B',
                           'score': 0.999999881,
                           'values': [1.0, 2.0, 3.0]},
                          {'id': 'A',
                           'score': 0.925820053,
                           'values': [0.99999994, 0.99999994, 0.99999994]}],
              'namespace': ''}]}

## Delete the Index
Delete the index once you are sure that you do not want to use it anymore. Once the index is deleted, you cannot use it again.

In [10]:
pinecone.delete_index(index_name)